In [1]:
%pip install azure-ai-ml
%pip install azure-identity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 4.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.8/314.8 kB 930.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 686.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 kB 659.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 661.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.5/284.5 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.3/770.3 kB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━

In [20]:
from azure.ai.ml import MLClient
from azure.identity import AzureCliCredential
from azure.ai.ml.entities import ManagedOnlineEndpoint

credential = AzureCliCredential()

ml_client = MLClient(
    credential=credential,
    subscription_id="05c54f8a-eea2-4804-9157-82e9d39f3598",
    resource_group_name="wav2vec2-rg",
    workspace_name="wav2vec2-aml",
)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [21]:
registry_ml_client = MLClient(credential=credential,registry_name="HuggingFace")

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [22]:
models = registry_ml_client.models.list(name="facebook-Wav2Vec2-Base-960h")

for m in models:
    print(m.name, m.version)


facebook-wav2vec2-base-960h 3
facebook-wav2vec2-base-960h 2
facebook-wav2vec2-base-960h 1


In [23]:
model_name = "facebook-Wav2Vec2-Base-960h"
model = registry_ml_client.models.get(name=model_name, version="3")

In [30]:
endpoint = ManagedOnlineEndpoint(
    name="hf-wav2vec2-endpoint-sti",
    auth_mode="key"
)

ml_client.online_endpoints.begin_create_or_update(endpoint).wait()
print("Endpoint created successfully")


Endpoint created successfully


In [3]:
from azure.ai.ml.entities import Environment

env = Environment(
    name="hf-wav2vec2-env",
    image="mcr.microsoft.com/azureml/curated/huggingface-transformers:latest",
)

In [2]:
from azure.ai.ml.entities import ManagedOnlineDeployment

demo_deployment = ManagedOnlineDeployment(
    name="hf-wav2vec2-deployment-sti",
    endpoint_name="hf-wav2vec2-endpoint-sti",
    model=model,  # pass the model object directly for registry models
    instance_type="Standard_DS2_v2",
    instance_count=1,
)
ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
print("Deployment created successfully")
ml_client.begin_create_or_update(endpoint).result()

NameError: name 'model' is not defined